<a href="https://colab.research.google.com/github/mtzig/LIDC_GDRO/blob/main/transfer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
!git clone https://github.com/mtzig/LIDC_GDRO.git
%cd /content/LIDC_GDRO

fatal: destination path 'LIDC_GDRO' already exists and is not an empty directory.
/content/LIDC_GDRO


In [141]:
!git pull

Already up to date.


In [142]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import NoduleDataset
from dataloaders import SubtypedDataLoader

In [143]:
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Using cpu")
    DEVICE = torch.device("cpu")

Good to go!


In [144]:
lidc_subtyped = pd.read_csv('./data/lidc_spic_subgrouped.csv', index_col=0)
lidc_subtyped.head(5)

,noduleID,malignancy,spiculation,malignancy_b,spiculation_b,subgroup
0,1,5.0,5.0,malignant,marked,marked_malignant
1,2,5.0,1.0,malignant,unmarked,unmarked_malignant
2,3,5.0,2.0,malignant,marked,marked_malignant
3,4,4.0,2.0,malignant,marked,marked_malignant
4,5,5.0,1.0,malignant,unmarked,unmarked_malignant


In [145]:
lidc_subtyped.groupby(['spiculation_b', 'malignancy_b']).size()

spiculation_b  malignancy_b
marked         benign          101
               malignant       407
unmarked       benign          725
               malignant       282
dtype: int64

In [146]:
def getNormed(this_array, this_min = 0, this_max = 255, set_to_int = True):
    
    rat = (this_max - this_min)/(this_array.max() - this_array.min())
    this_array = this_array * rat
    this_array -= this_array.min()
    this_array += this_min
    if set_to_int:
        return this_array.to(dtype= torch.int)
    return this_array

In [147]:
def getImages(image_folder):
    '''
        Input:
        image_folder: directory of the image files

        Output:
        m1: list of the labels encountered (1,2,4,5)
        m2: list of binary labels encountered (benign, malignant)
        diff: list of any nodes with discrepency to CSV labels

    '''
    
    train_img = []
    train_label = []
    train_spic_label = []
    marked_benign = []
    unmarked_benign = []
    
    marked_malignant = []
    unmarked_malignant = []

    

    lidc = pd.read_csv('./data/lidc_spic_subgrouped.csv')
    train_test = pd.read_csv('./data/lidc_train_test_split_stratified.csv')
    for dir1 in os.listdir(image_folder):
  
        if dir1 == 'Malignancy_3':
            continue

        for file in os.listdir(os.path.join(image_folder, dir1)):


            temp_nodule_ID = file.split('.')[0]
            subtype = lidc[lidc['noduleID']==int(temp_nodule_ID)]['subgroup'].iloc[0]
            malignancy = lidc[lidc['noduleID']==int(temp_nodule_ID)]['malignancy'].iloc[0]
            spiculation = lidc[lidc['noduleID']==int(temp_nodule_ID)]['malignancy'].iloc[0]
            
            train_type = train_test[train_test['noduleID'] ==int(temp_nodule_ID)]['dataset'].iloc[0]
            
            
            image = np.loadtxt(os.path.join(image_folder, dir1,file))
            image = torch.from_numpy(image).to(DEVICE)
            rgb_image = torch.stack((image,image,image), dim = 0)
            rgb_image = getNormed(rgb_image)
            rgb_image = rgb_image / 255 

            
            
            if train_type == 'train':
                train_img.append(rgb_image)
                train_label.append(torch.tensor(1).to(DEVICE).to(torch.float32) if malignancy > 3 else torch.tensor(0).to(DEVICE).to(torch.float32))
                train_spic_label.append(torch.tensor(1).to(DEVICE).to(torch.float32) if spiculation > 1 else torch.tensor(0).to(DEVICE).to(torch.float32))
                
                continue
            
            if subtype == 'marked_benign':
                image_array = marked_benign
            elif subtype == 'unmarked_benign':
                image_array = unmarked_benign
            elif subtype == 'marked_malignant':
                image_array = marked_malignant
            else:
                image_array = unmarked_malignant
            
            image_array.append(rgb_image)
 


    return train_img, train_label, train_spic_label, marked_benign, unmarked_benign, marked_malignant, unmarked_malignant


In [148]:
train_img, train_label, train_spic_label, marked_benign, unmarked_benign, marked_malignant, unmarked_malignant = getImages('./LIDC(MaxSlices)_Nodules(fixed)')

In [149]:
sum(train_spic_label)

tensor(952., device='cuda:0')

In [150]:
len(train_img)

1219

In [151]:
len(marked_benign)

17

In [152]:
len(unmarked_benign)

149

In [153]:
len(marked_malignant)

83

In [154]:
len(unmarked_malignant)

47

#model train!!!

In [155]:
from models import VGGNet
from loss import ERMLoss
from datasets import NoduleDataset
from dataloaders import InfiniteDataLoader

import train

In [156]:
model = VGGNet(device=DEVICE)

In [157]:
model

VGGNet(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): C

In [158]:
loss_fn = ERMLoss(model,torch.nn.functional.binary_cross_entropy_with_logits,{}) #torch.nn.CrossEntropyLoss()

In [159]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.005)


In [160]:
train_dataset = NoduleDataset(train_img, train_label) #train_spic_label

In [161]:
len(train_dataset)

1219

In [162]:
train_set, val_set = torch.utils.data.random_split(train_dataset, [1000, 219])

In [163]:
train_loader = InfiniteDataLoader(train_set, 1000, num_workers=0)

In [164]:
val_loader = InfiniteDataLoader(val_set, 219, num_workers = 0)

In [165]:
x,y = next(val_loader)

In [166]:
y.dtype

torch.float32

In [167]:
epochs = 40
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train.train(train_loader, model, loss_fn, optimizer, verbose=True)
                # train.test(test_dataloader, model)

    model.eval()
    with torch.no_grad():
      X,y = next(val_loader)
      results = torch.sigmoid(model(X))
      accuracy = torch.sum(torch.round(results) == y)/219
    model.train()
    print(f'cv accuracy {accuracy}')

Epoch 1/40
Average training loss: 0.6946945190429688
cv accuracy 0.611872136592865
Epoch 2/40
Average training loss: 0.677635908126831
cv accuracy 0.5205479264259338
Epoch 3/40
Average training loss: 0.6549263596534729
cv accuracy 0.6255707740783691
Epoch 4/40
Average training loss: 0.6415143013000488
cv accuracy 0.6986300945281982
Epoch 5/40
Average training loss: 0.6368833780288696
cv accuracy 0.7351598143577576
Epoch 6/40
Average training loss: 0.6449071168899536
cv accuracy 0.6392693519592285
Epoch 7/40
Average training loss: 0.6267386078834534
cv accuracy 0.6986300945281982
Epoch 8/40
Average training loss: 0.5968575477600098
cv accuracy 0.6712328195571899
Epoch 9/40
Average training loss: 0.6072192788124084
cv accuracy 0.7123287320137024
Epoch 10/40
Average training loss: 0.5864220261573792
cv accuracy 0.7168949246406555
Epoch 11/40
Average training loss: 0.5770047903060913
cv accuracy 0.7214611768722534
Epoch 12/40
Average training loss: 0.5620027184486389
cv accuracy 0.68036526

#marked_benign

In [168]:
len(marked_benign)

17

In [169]:
results = torch.sigmoid(model(torch.stack(marked_benign).to(DEVICE)))
accuracy = torch.sum(torch.round(results) == torch.zeros(len(marked_benign), device=DEVICE))/len(marked_benign)
accuracy

tensor(0.7059, device='cuda:0')

#unmarked_benign

In [170]:
len(unmarked_benign)

149

In [171]:
results = torch.sigmoid(model(torch.stack(unmarked_benign).to(DEVICE)))
accuracy = torch.sum(torch.round(results) == torch.zeros(len(unmarked_benign), device=DEVICE))/len(unmarked_benign)
accuracy

tensor(0.8523, device='cuda:0')

#marked_malignant



In [172]:
results = torch.sigmoid(model(torch.stack(marked_malignant).to(DEVICE)))
accuracy = torch.sum(torch.round(results) == torch.ones(len(marked_malignant), device=DEVICE))/len(marked_malignant)
accuracy

tensor(0.6386, device='cuda:0')

In [173]:
len(marked_malignant)

83

#unmarked malignant

In [174]:
len(unmarked_malignant)

47

In [175]:
results = torch.sigmoid(model(torch.stack(unmarked_malignant).to(DEVICE)))
accuracy = torch.sum(torch.round(results) == torch.ones(len(unmarked_malignant), device=DEVICE))/len(unmarked_malignant)
accuracy

tensor(0.5106, device='cuda:0')